Fluxo ELT(extração, transformação, carga):
 - Extração: Extraímos os dados da API OMDb e do CSV da Netflix.
 - Transformação: Integração dos dados, limpeza, padronização, enriquecimento e outras transformações.
 - Carga: Salvamos o arquivo final em formato Parquet.

In [16]:
from dotenv import load_dotenv
import os

# carregar variaveis do .env file
load_dotenv()

# acessar minha chave api
OMDB_API_KEY = os.getenv('OMDB_API_KEY')

# # Test to ensure the key is loaded (print only for debugging, avoid printing in production)
# print(OMDB_API_KEY)

In [17]:
import pandas as pd
import requests
import pyarrow as pa
import pyarrow.parquet as pq

# a) Integração: Função para extrair dados da API OMDb
def get_imdb_rating(title):
    url = f"http://www.omdbapi.com/?t={title}&apikey={OMDB_API_KEY}"
    response = requests.get(url)
    data = response.json()
    # Verifica se a resposta da API foi bem-sucedida
    if response.status_code == 200:
        return response.json()
    else:
        return None

# obter os detalhes de um filme específico
movie_details = get_imdb_rating("Inception")

# Exibir detalhes do filme
print(movie_details)


{'Title': 'Inception', 'Year': '2010', 'Rated': 'PG-13', 'Released': '16 Jul 2010', 'Runtime': '148 min', 'Genre': 'Action, Adventure, Sci-Fi', 'Director': 'Christopher Nolan', 'Writer': 'Christopher Nolan', 'Actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page', 'Plot': 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.', 'Language': 'English, Japanese, French', 'Country': 'United States, United Kingdom', 'Awards': 'Won 4 Oscars. 159 wins & 220 nominations total', 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '87%'}, {'Source': 'Metacritic', 'Value': '74/100'}], 'Metascore': '74', 'imdbRating': '8.8', 'imdbVotes': '2,602,766', 'imdbID'

In [2]:
# a) Integração: Carregar dados do CSV da Netflix
# Caminho do CSV
csv_file_path = 'netflix_titles.csv'

# Carregar o dataset
netflix_data = pd.read_csv(csv_file_path)

# Verificar as primeiras linhas do dataset
print(netflix_data.head())


  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

In [3]:
# a) Integração: Extrair dados da API OMDb e adicionar ao dataset
# Adicionar uma nova coluna para as classificações de filmes/séries obtidas pela API OMDb
netflix_data['imdb_rating'] = None

# Vamos iterar sobre as primeiras 10 linhas do dataset para buscar informações adicionais da API OMDb
for index, row in netflix_data.head(10).iterrows():
    movie_title = row['title']

    # Buscar detalhes do filme na API
    movie_details = get_imdb_rating(movie_title)

    if movie_details and 'imdbRating' in movie_details:
        # Atualizar o DataFrame com a classificação IMDb
        netflix_data.at[index, 'imdb_rating'] = movie_details['imdbRating']
    else:
        netflix_data.at[index, 'imdb_rating'] = 'N/A'  # Caso não encontre o filme na API

# Exibir as primeiras linhas com a nova coluna 'imdb_rating'
print(netflix_data.head(10))

  show_id     type                             title  \
0      s1    Movie              Dick Johnson Is Dead   
1      s2  TV Show                     Blood & Water   
2      s3  TV Show                         Ganglands   
3      s4  TV Show             Jailbirds New Orleans   
4      s5  TV Show                      Kota Factory   
5      s6  TV Show                     Midnight Mass   
6      s7    Movie  My Little Pony: A New Generation   
7      s8    Movie                           Sankofa   
8      s9  TV Show     The Great British Baking Show   
9     s10    Movie                      The Starling   

                        director  \
0                Kirsten Johnson   
1                            NaN   
2                Julien Leclercq   
3                            NaN   
4                            NaN   
5                  Mike Flanagan   
6  Robert Cullen, José Luis Ucha   
7                   Haile Gerima   
8                Andy Devonshire   
9                 Theod

In [4]:
# b) Limpeza: Remover entradas com título ou país ausente
netflix_data.dropna(subset=['title', 'country', 'release_year'], inplace=True)

# Remover valores de imdb_rating ausentes
netflix_data.dropna(subset=['imdb_rating'], inplace=True)

# Remover duplicatas
netflix_data.drop_duplicates(inplace=True)

# Exibir informações básicas após a limpeza
print(netflix_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 0 to 9
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       6 non-null      object
 1   type          6 non-null      object
 2   title         6 non-null      object
 3   director      4 non-null      object
 4   cast          5 non-null      object
 5   country       6 non-null      object
 6   date_added    6 non-null      object
 7   release_year  6 non-null      int64 
 8   rating        6 non-null      object
 9   duration      6 non-null      object
 10  listed_in     6 non-null      object
 11  description   6 non-null      object
 12  imdb_rating   6 non-null      object
dtypes: int64(1), object(12)
memory usage: 672.0+ bytes
None


In [5]:
# 3. c) Padronização: Padronizar os nomes dos países para caixa alta
netflix_data['country'] = netflix_data['country'].str.upper()

# Padronizar a classificação IMDb (normalizando para uma escala de 0 a 10)
netflix_data['imdb_rating'] = netflix_data['imdb_rating'].astype(float)

# Exibir as primeiras linhas com as alterações
print(netflix_data.head(10))

  show_id     type                          title         director  \
0      s1    Movie           Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show                  Blood & Water              NaN   
4      s5  TV Show                   Kota Factory              NaN   
7      s8    Movie                        Sankofa     Haile Gerima   
8      s9  TV Show  The Great British Baking Show  Andy Devonshire   
9     s10    Movie                   The Starling   Theodore Melfi   

                                                cast  \
0                                                NaN   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
7  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...   
8  Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...   
9  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...   

                                             country          date_added  \
0                                      U

In [6]:
from datetime import datetime

# 3. d) Enriquecimento: Adicionar uma coluna "idade do conteúdo"
current_year = datetime.now().year
netflix_data['content_age'] = current_year - netflix_data['release_year']
# Exibir as primeiras linhas com as alterações
print(netflix_data.head(10))

  show_id     type                          title         director  \
0      s1    Movie           Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show                  Blood & Water              NaN   
4      s5  TV Show                   Kota Factory              NaN   
7      s8    Movie                        Sankofa     Haile Gerima   
8      s9  TV Show  The Great British Baking Show  Andy Devonshire   
9     s10    Movie                   The Starling   Theodore Melfi   

                                                cast  \
0                                                NaN   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...   
7  Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...   
8  Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...   
9  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...   

                                             country          date_added  \
0                                      U

In [9]:
# 3. e) Filtros e Seleções: Filmes e séries com IMDb acima de 7 e lançados nos últimos 10 anos
filtered_data = netflix_data[
    (netflix_data['imdb_rating'] > 7.0) &
    (netflix_data['release_year'] >= current_year - 10)
]

# Exibir o resultado
print(filtered_data)

  show_id     type                          title         director  \
0      s1    Movie           Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show                  Blood & Water              NaN   
4      s5  TV Show                   Kota Factory              NaN   
8      s9  TV Show  The Great British Baking Show  Andy Devonshire   

                                                cast         country  \
0                                                NaN   UNITED STATES   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...    SOUTH AFRICA   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...           INDIA   
8  Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...  UNITED KINGDOM   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
4  September 24, 2021          2021  TV-MA  2 Seasons   
8  September 24, 2021          2021  TV-14  9 Seasons   

  

In [13]:
# 4. Carga: Salvar os dados filtrados e transformados em formato Parquet
# Definir o caminho de saída para o arquivo Parquet
output_file = 'netflix_data_transformed.parquet'

# Converter para PyArrow Table
table = pa.Table.from_pandas(filtered_data)

# Salvar como arquivo Parquet
pq.write_table(table, output_file)

print(f"Arquivo Parquet salvo em: {output_file}")

Arquivo Parquet salvo em: netflix_data_transformed.parquet
